In [3]:
import torch
import os
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2

BASE_DIR = os.getcwd()
path_to_imgs = 'sample_images/images'

In [11]:
def adjust_saturation(image, target_saturation):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
    mean_saturation = hsv[:, :, 1].mean() / 255.0
    adjustment_ratio = target_saturation / mean_saturation
    hsv[:, :, 1] = np.clip(hsv[:, :, 1] * adjustment_ratio, 0, 255)
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2RGB)

In [ ]:
def run_images(model: YOLO, image_folder: str, confidence: float=0.5, target_saturation: float=0.42) -> None:
    path_to_imgs = image_folder+'/images'
    names = model.names

    for file in os.listdir(path_to_imgs):

        if not file.lower().endswith(('.jpg', '.png', '.bmp', 'jpeg')):
            continue
        
        list_of_centers = []
        image_path = os.path.join(path_to_imgs, file)
        image = cv2.imread(image_path)
        image = adjust_saturation(image, target_saturation)

        height, width, _ = image.shape

        results = model.predict(image, conf=confidence)[0]

        # Создаем копию изображения для рисования
        image_copy = image.copy()

        # Получаем все детекции
        detections = results.boxes.xyxy.cpu().numpy()
        classes = results.boxes.cls.cpu().numpy()
        

        # Цикл по всем детекциям
        for i, detection in enumerate(detections):
            predict_cls = names[classes[i]]
            if predict_cls.lower() != 'apple':
                continue
            # Получаем координаты центра bounding box
            x_center = int((detection[0] + detection[2]) / 2)
            y_center = int((detection[1] + detection[3]) / 2)
            list_of_centers.append((x_center, y_center))
            
            # Рисуем центр bounding box на изображении
            cv2.circle(image_copy, (x_center, y_center), 25, (255, 0, 0), -1)

        

        formatted_rel_list_of_centers = [ '%.2f' % (elem[0]/width) + ', ' + '%.2f' % (elem[1]/height) for elem in list_of_centers ]
        print(f'List of centers for {file}\nAbsolute values {list_of_centers}\nRelative values {formatted_rel_list_of_centers}')

        # Проверяем, соответствуют ли предсказания меткам
        label_path = os.path.join(image_folder+'/labels', os.path.basename(file)[0]+'.txt')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                
                labels = f.readlines()
                true_predictions = 0

                
                
                for center in list_of_centers:
                    
                    closest_dist = float("inf")
                    closest_box = None  
                    for i, label in enumerate(labels):
                        _, x, y, w, h = map(float, label.split())
                        x_center_label = int(x * width)
                        y_center_label = int(y * height)
                        x_left_label = int((x-w)*width) if x-w >= 0 else 0
                        x_right_label = int((x+w)*width) if x+w <= width else width
                        y_top_label = int((y-h)*height) if y-h >= 0 else 0
                        y_down_label = int((y+h)*height) if y+h <= height else height
                        cv2.circle(image_copy, (x_center_label, y_center_label), 25, (0, 0, 255), -1)




                        if (
                                np.sqrt((center[0] - x_center_label)**2 + (center[1] - y_center_label)**2) < closest_dist and
                                x_left_label <= center[0] <= x_right_label and
                                y_top_label <= center[1] <= y_down_label
                            ):
                            
                            closest_dist = np.sqrt((center[0] - x_center_label)**2 + (center[1] - y_center_label)**2)
                            closest_box = i

                    if closest_box is not None:
                        true_predictions +=1
                

                print(f'True predictions for {file}: {true_predictions} out of {len(labels)}, accuracy: {true_predictions/len(labels):.2f}')
        else:
            print(f'No label file found for {file}')


        # Отображаем измененное изображение
        image_copy = cv2.resize(image_copy, (640, 448))
        plt.imshow(cv2.cvtColor(image_copy, cv2.COLOR_BGR2RGB))
        plt.title('Image with centers')
        plt.show()

In [9]:
model = YOLO('apple_detection/train3/weights/best.pt')
validation_results = model.val(data="sample_images/data.yaml", imgsz=640, conf=0.5, device="0")

Ultralytics YOLOv8.2.68  Python-3.12.2 torch-2.4.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning F:\PycharmProjects\randoms\test_PeK\sample_images\labels.cache... 6 images, 0 backgrounds, 0 corrupt: 100%|██████████| 6/6 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<00:00,  3.99s/it]


                   all          6        101      0.854      0.347      0.616      0.273
Speed: 7.0ms preprocess, 30.6ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\val2


In [3]:
import cv2
import os
import numpy as np

def average_saturation(folder_path):
    """
    Calculates the average saturation for all images in the specified folder.
    Parameters:
    folder_path (str): The path to the folder containing the images.
    
    Returns:
    None
    """
    total_saturation = 0 
    num_images = 0 

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img = cv2.imread(os.path.join(folder_path, filename)) 
            img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 
            saturation = np.mean(img[:,:,1])/255 
            total_saturation += saturation  
            num_images += 1  
    
    if num_images > 0:
        average_saturation = total_saturation / num_images  
        print(f'Average saturation across all images in the folder: {average_saturation}')  
    else:
        print('No images found in the folder')

average_saturation('This-Apple-1/train/images')


Average saturation across all images in the folder: 0.486873031231777


In [5]:
model = YOLO('apple_detection/train4/weights/best.pt')
run_images(model, path_to_imgs, target_saturation=0.486873031231777)

SyntaxError: '[31m[1muse_sahi[0m' is not a valid YOLO argument. Similar arguments are i.e. ['close_mosaic=10'].

    Arguments received: ['yolo', '--f=c:\\Users\\Spidrre\\AppData\\Roaming\\jupyter\\runtime\\kernel-v391d19e0a2867ea9414581916ef55d8a2252f6feb.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of {'pose', 'segment', 'obb', 'detect', 'classify'}
                MODE (required) is one of {'track', 'train', 'export', 'val', 'predict', 'benchmark'}
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolov8n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at image size 320:
 

In [43]:
# Загрузка модели
from sahi import AutoDetectionModel
from sahi.predict import get_prediction, get_sliced_prediction
import math
from PIL import Image



#model_path = 'apple_detection/100m3/weights/best.pt'
model_path = 'yolov8m.pt'


detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",
    model_path=model_path,
    #model_path='yolov8m.pt',
    confidence_threshold=0.7,
    device="cuda:0",  # or 'cuda:0'
)

model = YOLO(model_path)

path_to_imgs = 'sample_images/images'
for file in os.listdir(path_to_imgs):

        if not file.lower().endswith(('.jpg', '.png', '.bmp', 'jpeg')):
            continue
        
        list_of_centers = []
        image_path = os.path.join(path_to_imgs, file)
        image = cv2.imread(image_path)
        
        img_height, img_width, _ = image.shape

        if img_height > 2000 or img_width > 2000:

            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            window_size = 1024

            height_extra = math.floor((math.ceil(img_height / window_size) * window_size - img_height) / math.ceil(img_height / window_size)) / window_size
            width_extra = math.floor((math.ceil(img_width / window_size) * window_size - img_width) / math.ceil(img_width / window_size)) / window_size
            y_overlap = height_extra
            x_overlap = width_extra

            result = get_sliced_prediction(
                image,
                detection_model,
                slice_height=window_size,
                slice_width=window_size,
                overlap_height_ratio=y_overlap,
                overlap_width_ratio=x_overlap,
                postprocess_type='NMS',
                postprocess_match_metric='IOU',
                postprocess_match_threshold=0.2,
            )

            result.export_visuals(export_dir=f"demo/test", file_name=f"{file}_result.png")


        else:
             #image = adjust_saturation(image, 0.486873031231777)
             results = model(image, conf=0.4, iou=0.7)
             # Сохранить результаты извлечения модели yolov8 для обнаружения объектов как изображение
             for result in results:
                result.save(f"demo/test/{file}_result.png")
             
             

Performing prediction on 24 slices.
Performing prediction on 24 slices.
Performing prediction on 24 slices.

0: 384x640 5 apples, 6.0ms
Speed: 2.0ms preprocess, 6.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 apples, 6.9ms
Speed: 2.1ms preprocess, 6.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 apples, 7.0ms
Speed: 2.0ms preprocess, 7.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
